In [15]:
import pandas as pd

In [16]:
df = pd.read_csv("Data/Hotel Reservations.csv")

In [17]:
df.head(5)

,Booking_ID,no_of_adults,no_of_children,no_of_weekend_nights,no_of_week_nights,type_of_meal_plan,required_car_parking_space,room_type_reserved,lead_time,arrival_year,arrival_month,arrival_date,market_segment_type,repeated_guest,no_of_previous_cancellations,no_of_previous_bookings_not_canceled,avg_price_per_room,no_of_special_requests,booking_status
0,INN00001,2,0,1,2,Meal Plan 1,0,Room_Type 1,224,2017,10,2,Offline,0,0,0,65.00,0,Not_Canceled
1,INN00002,2,0,2,3,Not Selected,0,Room_Type 1,5,2018,11,6,Online,0,0,0,106.68,1,Not_Canceled
2,INN00003,1,0,2,1,Meal Plan 1,0,Room_Type 1,1,2018,2,28,Online,0,0,0,60.00,0,Canceled
3,INN00004,2,0,0,2,Meal Plan 1,0,Room_Type 1,211,2018,5,20,Online,0,0,0,100.00,0,Canceled
4,INN00005,2,0,1,1,Not Selected,0,Room_Type 1,48,2018,4,11,Online,0,0,0,94.50,0,Canceled


In [18]:
# show rows and columns of dataset
df.shape

(36275, 19)

In [19]:
# find unique values 
df.nunique()

Booking_ID                              36275
no_of_adults                                5
no_of_children                              6
no_of_weekend_nights                        8
no_of_week_nights                          18
type_of_meal_plan                           4
required_car_parking_space                  2
room_type_reserved                          7
lead_time                                 352
arrival_year                                2
arrival_month                              12
arrival_date                               31
market_segment_type                         5
repeated_guest                              2
no_of_previous_cancellations                9
no_of_previous_bookings_not_canceled       59
avg_price_per_room                       3930
no_of_special_requests                      6
booking_status                              2
dtype: int64

In [20]:
# detect missing values
missing_values = df.isnull().sum()
missing_values = missing_values[missing_values > 0]

# display missing values
if not missing_values.empty:
    print("Number of missing values for each feature:")
    print(missing_values)
else:
    print("No missing values detected in the dataset.")

No missing values detected in the dataset.


In [21]:
# deal with special characters on categorical data
cat_columns = df.select_dtypes(['object']).columns
for col in cat_columns:
    # remove white spaces, lower
    df[col] = df[col].str.strip()
    df[col] = df[col].str.lower()

    # replace all special characters except alphanumerics, spaces, and dashes
    df[col] = df[col].str.replace('[^a-zA-Z0-9\s_]', '', regex=True)

In [22]:
# drop duplicate rows
df.drop_duplicates(inplace=True)

In [23]:
# check if 'no_of_adults', 'no_of_children', 'no_of_weekend_nights', 'no_of_week_nights', 'lead_time',
# 'no_of_previous_cancellations', 'no_of_previous_bookings_not_canceled', 'no_of_special_requests' 
# are non-negative integer
columns_to_check = ['no_of_adults', 'no_of_children', 'no_of_weekend_nights', 'no_of_week_nights', 
                    'lead_time', 'no_of_previous_cancellations', 'no_of_previous_bookings_not_canceled', 
                    'no_of_special_requests']

for col in columns_to_check:
    if df[col].dtype == 'int' and (df[col] >= 0).all():
        print(f"{col} contains only non-negative integer.")
    else:
        print(f"{col} contains other data type.")

no_of_adults contains other data type.
no_of_children contains other data type.
no_of_weekend_nights contains other data type.
no_of_week_nights contains other data type.
lead_time contains other data type.
no_of_previous_cancellations contains other data type.
no_of_previous_bookings_not_canceled contains other data type.
no_of_special_requests contains other data type.


In [24]:
# check if 'required_car_parking_space', 'repeated_guest', contains binary values
binary_columns = ['required_car_parking_space', 'repeated_guest']
for col in binary_columns:
    if set(df[col].unique()) == {0, 1}:
        print(f"{col} contains binary values.")
    else:
        print(f"{col} does not contain binary values.")

required_car_parking_space contains binary values.
repeated_guest contains binary values.


In [25]:
# check if the columns 'arrival_year', 'arrival_month', and 'arrival_date' contain valid dates
df['arrival_y_m_d'] = df['arrival_year'].astype(str) + '-' + df['arrival_month'].astype(str) + '-' + df['arrival_date'].astype(str)

# convert the combined column to datetime
df['arrival_y_m_d'] = pd.to_datetime(df['arrival_y_m_d'], errors='coerce')

# check if there are any NaT (Not a Time) values, indicating invalid dates
if df['arrival_y_m_d'].isnull().any():
    print("Invalide date exits.")
else:
    print("All dates are valid.")

# remove invalid dates
df = df[~df['arrival_y_m_d'].isnull()]

# check again
if df['arrival_y_m_d'].isnull().any():
    print("Invalide date exits.")
else:
    print("All dates are valid.")
    
df.drop('arrival_y_m_d', axis=1, inplace=True)

Invalide date exits.
All dates are valid.


In [26]:
# check the unique values in categorical variables 
cat_columns = df.select_dtypes(['object']).columns[1:] # exclude booking_id
for col in cat_columns:
    unique_values = df[col].unique()
    print(f"Unique values in column '{col}': {unique_values}")

Unique values in column 'type_of_meal_plan': ['meal plan 1' 'not selected' 'meal plan 2' 'meal plan 3']
Unique values in column 'room_type_reserved': ['room_type 1' 'room_type 4' 'room_type 2' 'room_type 6' 'room_type 5'
 'room_type 7' 'room_type 3']
Unique values in column 'market_segment_type': ['offline' 'online' 'corporate' 'aviation' 'complementary']
Unique values in column 'booking_status': ['not_canceled' 'canceled']


In [27]:
df.to_csv('Data/cleaned.csv', index=False)